# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [12]:
# CID : 카테고리(소설 : 1, 에세이 :55889 , 자기계발 : 336)
#results = []

for page in range(632, 651): # 페이지 입력
    url = "https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail\
    &PublishMonth=0&SortOrder=2&page={}&Stockstatus=0&PublishDay=84&CustReviewRankStart=0&CustReviewCountStart=0&PriceFilterMax=-1\
    &CID=1&SearchOption=".format(page)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('ul > li > a.bo3')
    links = []
    for book_link in book_links:
        a= book_link['href']
        links.append(a)
    for link in links:
        driver.get(link)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            ISBN = soup.select('div.conts_info_list1 > ul > li')
            ISBN = ISBN[-1].text[-13:]
        except:
            pass

        #data = [ISBN]
        results.append(ISBN)

In [10]:
results[-1]

'9788972207283'

In [ ]:
import pickle
ISBN141 = pd.DataFame(results)
ISBN141.to_pickle('ISBN141.plk')

# 2. API 조회하기

In [6]:
import requests
import pandas as pd
import json

In [7]:
# API 호출키
KEY = 'ttbsueji951535001'

In [13]:
url_look_list = []

for isbn in results:
    url_look = 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={}&itemIdType=isbn13&ItemId={}&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList'.format(KEY, isbn)
    url_look_list.append(url_look)
print(url_look_list)

['https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788936463328&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788926313695&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788925536453&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&ItemId=9788925538228&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbsueji951535001&itemIdType=isbn13&It

In [14]:
items = []
lists = []
for url_look in url_look_list:
    try:
        response = requests.get(url_look)
        text = json.loads(response.text)
        #time.sleep(1)
        title = text['item'][0]['title']
        author = text['item'][0]['author']
        pubDate = text['item'][0]['pubDate']
        description = text['item'][0]['description']
        isbn13 = text['item'][0]['isbn13']
        categoryID = text['item'][0]['categoryId']
        categoryName = text['item'][0]['categoryName']
        salesPoint = text['item'][0]['salesPoint']
        adult = text['item'][0]['adult']
        rank = text['item'][0]['customerReviewRank']
        fullDes = text['item'][0]['fullDescription']
        fullDes2 = text['item'][0]['fullDescription2']
        story = text['item'][0]['subInfo']['story']
        try:
            reviewrank = text['item'][0]['reviewList'][0]['reviewRank']
            reviewtitle = text['item'][0]['reviewList'][0]['title']
            review_w = text['item'][0]['reviewList'][0]['writer']
        except:
            reviewrank = 0
            reviewtitle = 0
            review_w = 0
        lists.append([title, author, pubDate, description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w])
    except:
        pass
cols = ['title', 'author', 'pubDate','description','isbn13','categoryID','categoryName','salesPoint','adult','rank','fullDes','fullDes2','story','reviewrank','reviewtitle','review_w']
df = pd.DataFrame(lists, columns = cols)

In [15]:
import pickle
df.to_pickle('novel_601-650_.pkl', protocol=4)

In [16]:
df

,title,author,pubDate,description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w
0,문학과 시대현실 - 염무웅 평론집,염무웅 (지은이),2010-11-06,1964년 등단한 이래 &lt;혼돈의 시대에 구상하는 문학의 논리&gt;(1995)...,9788936463328,51305,국내도서>소설/시/희곡>문학의 이해>한국문학론>한국비평론,68,False,9,1964년 등단한 이래 &lt;혼돈의 시대에 구상하는 문학의 논리&gt;(1995)...,1964년 등단한 이래 『혼돈의 시대에 구상하는 문학의 논리』(1995)까지 세 권...,,10,[마이리뷰]문학과 시대현실> 진정한 문..,책과의일상
1,시간상인 1 - J Novel,미즈이치 케이 (지은이),2010-08-11,,9788926313695,51105,국내도서>소설/시/희곡>라이트 노벨>제이(J) 노벨,68,False,8,"2권<br>불로불사(不老不死)를 필요로 하는 사람 앞에, 시간상인의 조수는 하얀 고...",,,4,[100자평]Aㅏ..... 이런 부류는 작가의 필력이 특히나 중요한...,hyun
2,가야산 정진불 2 - 혜암 큰스님 이야기,"정찬주 (지은이), 유동영 (사진)",2010-05-05,제23회 동국문학상 수상작. <가야산 정진불>은 성철 스님과 일타 스님의 생애를 소...,9788925536453,50993,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,68,False,0,2010년 제23회 동국문학상 수상작. <가야산 정진불>은 성철 스님과 일타 스님의...,<b>제23회 동국문학상 수상작 <BR>\n정찬주 구도 소설 혜암 큰스님 이야기<...,,0,0,0
3,사명대사 일본탐정기,박덕규 (지은이),2010-04-30,소설가 박덕규가 6년 만에 발표하는 장편소설. 이 작품은 조선 중기를 살았던 당대 ...,9788925538228,51123,국내도서>소설/시/희곡>역사소설>한국 역사소설,68,False,8,소설가 박덕규가 6년 만에 발표하는 장편소설. 이 작품은 조선 중기를 살았던 당대 ...,<b>“묘당에 세 정승이 있다 하지 마라<BR>\n조선의 운명은 한 승려에 달렸노라...,,8,[마이리뷰]사명대사 일본탐정기,햇살찬란
4,서른 개의 문을 지나온 사람,하창수 (지은이),2010-04-30,1991년 장편 <돌아서지 않는 사람들>로 한국일보문학상을 수상한 하창수의 소설집....,9788932020457,50993,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,68,False,0,1991년 장편 <돌아서지 않는 사람들>로 한국일보문학상을 수상한 하창수의 소설집....,,,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,기억을 잇다,소재원 (지은이),2017-06-07,소재원 장편소설. 아버지 이야기이다. 아버지의 무조건적인 희생이나 사랑보다는 보편적...,9788954437813,50993,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,56,False,9,소재원 장편소설. 아버지 이야기이다. 아버지의 무조건적인 희생이나 사랑보다는 보편적...,<b>『소원』 『터널』 『균』<BR>\n힘없고 외로운 사람들의 눈물을 대변하는 작가...,치매 판정을 받은 노년의 아버지와 명예퇴직을 당한 중년의 아들이 시간차를 두고 같은...,10,[마이리뷰][마이리뷰] 기억을 잇다,물고구마
2482,"도련님, 아프시면 수프라도 좀 드세요",최철호 (지은이),2017-05-19,서울시 관악구 봉천동에서 성장한 한 사내가 자기가 겪은 유년 시절의 추억을 찹쌀 호...,9791155310847,50993,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,56,False,10,서울시 관악구 봉천동에서 성장한 한 사내가 자기가 겪은 유년 시절의 추억을 찹쌀 호...,<b>69년생 최철호들 ― 변두리 삶들이 펼치는 리얼 7080 버라이어티</b><B...,,10,[100자평]68년생이라? 조금은 맛봤겠다. 대개가 비슷한 삶을 살...,역사smof
2483,[POD] 심리로 봉다방 - 심리학 오타쿠의 다방 창업기,왕고래 (지은이),2017-05-14,"왕고래 소설. 이론과 실전의 차이, 서로 다른 성향에서 일어나는 해프닝, 은은하게 ...",9791127215859,50993,국내도서>소설/시/희곡>한국소설>2000년대 이후 한국소설,56,False,0,"왕고래 소설. 이론과 실전의 차이, 서로 다른 성향에서 일어나는 해프닝, 은은하게 ...","실패 수집가, 대범한 행동파, 여사장, 봉순<BR>\n심리학 오타쿠, 소심한 전략가...",,0,0,0
2484,오페라의 유령 1 (미니북),"가스통 르루 (지은이), 베스트트랜스 (옮긴이)",2017-05-10,"가스통 르루의 대표작. 파리 오페라극장을 배경으로 공포와 미스터리, 목숨을 건 사랑...",9791159034831,50921,국내도서>소설/시/희곡>프랑스소설,56,False,0,‘더클래식 세계문학 컬렉션 미니북 도네이션’은 기존에 출간해왔던 ‘더클래식 세계문학...,<B>신이 아름다운 모습을 빼앗아 버린 음악의 천사!<BR>\n세계 4대 뮤지컬 <...,,0,0,0


In [11]:
pd.read_pickle('essay.pkl')

,title,author,pubDate,description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w
0,시맨틱 에러 포토에세이,"왓챠, 래몽래인 (지은이)",2022-03-20,왓챠 오리지널 드라마 ‘시맨틱 에러’ 포토에세이. 베스트셀러이자 스테디셀러인 원작의...,9791192186313,51376,국내도서>에세이>사진/그림 에세이,242520,False,10,왓챠 오리지널 드라마 ‘시맨틱 에러’ 포토에세이. 베스트셀러이자 스테디셀러인 원작의...,<b>론칭 즉시 왓챠 1위! <BR>\n전체 ott 콘텐츠 트렌드 탑2 등극<BR>...,,10,[100자평]안봤지만 만점갈겨 안봐도 극락...♡ 초판본아니면죽음을,뱅슈리a
1,"물고기는 존재하지 않는다 - 상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기","룰루 밀러 (지은이), 정지인 (옮긴이)",2021-12-17,집착에 가까울 만큼 자연계에 질서를 부여하려 했던 19세기 어느 과학자의 삶을 흥미...,9791189327156,51002,국내도서>과학>기초과학/교양과학,325120,False,9,‘방송계의 퓰리처상’으로 불리는 피버디상(Peabody Awards)을 수상한 과학...,"<b>《워싱턴포스트》, 《북라이엇》, 《내서널퍼블릭라디오NPR》, 《시카고 트리뷴》...",,4,[마이리뷰]잘 쓴 책이지만 좋은 과학책은 ..,mogulkor
2,오래되고 멋진 클래식 레코드,"무라카미 하루키 (지은이), 홍은주 (옮긴이)",2022-03-25,"본업인 소설가 외에도 사시사철 음악과 함께하는 애호가, 눈에 들어온 것은 저도 모르...",9788954685672,51373,국내도서>에세이>외국에세이,45050,False,9,"본업인 소설가 외에도 사시사철 음악과 함께하는 애호가, 눈에 들어온 것은 저도 모르...","<b>● “자랑할 정도는 아니지만, 이런 걸 모았답니다.”<BR>\n 60년째 ...",,10,[100자평]기다리는 시간도 즐겁다. 빨리 오기를,마젠타
3,H마트에서 울다,"미셸 자우너 (지은이), 정혜윤 (옮긴이)",2022-02-28,그래미 어워드 후보에 두 번 오른 인디 팝 밴드 재패니즈 브렉퍼스트의 가수이자 기타...,9788954683371,51373,국내도서>에세이>외국에세이,34810,False,10,그래미 어워드 후보에 두 번 오른 인디 팝 밴드 재패니즈 브렉퍼스트의 가수이자 기타...,<b>“엄마가 이제 내 곁에 없는데 내가 한국인일 수 있을까?”<BR>\n<BR>\...,,10,[100자평]지난해 제가 경험했던 무수한 감정들과 다시 만나야 하는...,riri
4,나에게 고맙다 - 30만 부 기념 전면개정판,전승환 (지은이),2022-02-18,2016년 출간 이후 30만 부 넘게 판매된 힐링 에세이 &lt;나에게 고맙다&gt...,9791191891072,51371,국내도서>에세이>한국에세이,26825,False,10,2016년 출간 이후 30만 부 넘게 판매된 힐링 에세이 &lt;나에게 고맙다&gt...,<b>★ ★ ★ ★ ★<BR>\n200만 독자가 사랑하는 '책 읽어주는 남자' 전승...,,10,[100자평]정말 예쁜 책. 어느 장을 펼쳐도 예쁜 모습을 하고 있...,chanel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1944,호찌냥찌 - 야생에서 쫓겨난 호랑이 호찌와 유쾌한 일곱 고양이들의 한집 살림 이야기,Grace J(정하나) (지은이),2019-08-23,"어느 날 우연히 숲속을 지나가는데, 아늑한 오두막집 앞에서 말하는 고양이와 사과를 ...",9791189969196,51376,국내도서>에세이>사진/그림 에세이,1623,False,10,"어느 날 우연히 숲속을 지나가는데, 아늑한 오두막집 앞에서 말하는 고양이와 사과를 ...",<b>〈라이온 킹〉만큼 경이롭고 감동적인 호랑이 삼촌 이야기<BR>\n“어른이 되어...,,10,[100자평]새로운 이야기 읽고 샀는데 역시 좋아요! 다음 시리즈도...,plum
1945,스위스의 고양이 사다리,"브리기테 슈스터 (지은이), 김목인 (옮긴이)",2021-01-15,애묘인 인구가 다수에 속하는 스위스에서 쉽게 마주칠 수 있는 ‘고양이 사다리’의 다...,9788986022285,51063,국내도서>예술/대중문화>사진>사진집,1620,False,8,애묘인 인구가 다수에 속하는 스위스에서 쉽게 마주칠 수 있는 ‘고양이 사다리’의 다...,애묘인 인구가 다수에 속하는 스위스에서 쉽게 마주칠 수 있는 ‘고양이 사다리’의 다...,,6,[마이리뷰]스위스의 고양이 사다리,dollc
1946,나는 당신이 살았으면 좋겠습니다 - 조울병 의사가 들려주는 조울병 이야기,안경희 (지은이),2018-02-08,조울병으로 '사회적 자살'에 이르렀던 저자가 자신의 경험을 바탕으로 쓴 에세이다. ...,9791187192787,51371,국내도서>에세이>한국에세이,1619,False,9,조울병 의사가 말하는 조울병에 관한 모든 것. 조울병으로 '사회적 자살'에 이르렀던...,<b>나는 왜 이렇게 감정 기복이 심할까? <BR>\n매일매일 널뛰는 감정으로 힘든...,,6,[100자평]저자의 진솔한 마음이 담긴 도서,stepout88
1947,우아한 가난의 시대 - 2020 문학나눔 선정도서,김지선 (지은이),2020-07-01,"""생각해 보면 지금까지는 엄살이었던 거고, 앞으로는 실제로 가난해질 확률이 너무나 ...",9791163641179,51371,국내도서>에세이>한국에세이,1617,False,9,<b>한국문화예술위원회 2020 하반기 문학나눔 도서보급사업에 선정된 도서입니다.<...,,,10,[마이리뷰]풍요로움과 가난함의 차이란,가필드


In [12]:
1949 / 50

38.98

In [ ]:
440 * 50